In [ ]:
day_obs = "2025-06-29"
#day_obs = "yesterday"
show_joined = False

# Identify potential bad images for {{ params.day_obs }}

This notebook uses information from the ConsDb and the EFD to try to identify potential bad visits.

Visits acquired from the FeatureBasedScheduler are tracked in the observatory system through several stages:
* a target event is issued when the visit is requested
* when this requested target is next to be observed, a nextvisit event is issued
* an observation event is issued when the script to acquire the event is registered as successful by the Scheduler
* as the visit is processed and metadata lands in the ConsDb, visit information including zeropoints measured by Rapid Analysis (RA) become available

These can be linked: target + observation are linked by `target_id`, target + nextvisit are linked by the `script sal index`, and nextvisit + consdb visit can be linked by `group_id`.

If the target is requested and visits are acquired, but the observation is never registered, then the script may have been interrupted  by a fault which would also cause problems for the visit.

If the zeropoint reported by RA is significantly far from the predicted zeropoint (currently using an ad-hoc per-visit magnitude cutoff), then it is likely that there were problems with the visit.

In each of these cases, we can identify the potentially troublesome visits and then check by eye in RubinTV -- links are provided to the full mosaic and the witness detector (at USDF).

---------

In [ ]:
!pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git --no-deps  > /dev/null 2>&1

In [ ]:
import rubin_nights
print('rubin_nights version', rubin_nights.__version__)

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
#from astroplan import Observer

from rubin_nights import connections, scriptqueue
import rubin_nights.dayobs_utils as rn_dayobs
from rubin_nights.targets_and_visits import targets_and_visits
    
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

import logging
logging.getLogger('rubin_nights').setLevel(logging.INFO)

----------

In [ ]:
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    tokenfile = '/Users/lynnej/.lsst/usdf_rsp'
    site = 'usdf'
else:
    tokenfile = None
    site = None
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)

In [ ]:
if day_obs.lower() == "today":
    day_obs = rn_dayobs.today_day_obs()
elif day_obs.lower() == "yesterday":
    day_obs = rn_dayobs.yesterday_day_obs()
else:
    try:
        int(day_obs)
        day_obs = rn_dayobs.day_obs_int_to_str(day_obs)
    except ValueError:
        pass
    
day_obs_time = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
#observer = Observer.at_site('Rubin')
#sunset = Time(observer.sun_set_time(day_obs_time, which='next', horizon=-10*u.deg), format='jd')
#sunrise = Time(observer.sun_rise_time(day_obs_time, which='next', horizon=-10*u.deg), format='jd')
sunset = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
sunrise = Time(f"{day_obs}T12:00:00", format='isot', scale='tai') + TimeDelta(1, format='jd')
print(f"Checking visits on {day_obs}, from {sunset.iso} to {sunrise.iso}")
print(f"Time of notebook execution {Time.now().iso}")

endpoints = connections.get_clients(tokenfile, site)
vt, cols, to, nv, visits = targets_and_visits(sunset, sunrise, endpoints)
print(f"Found {len(vt)} targets->visits.")
print(f"Visit metadata included {visits.scheduler_note.unique()}, {visits.target_name.unique()}")

In [ ]:
uri = "https://raw.githubusercontent.com/lsst-dm/excluded_visits/" "refs/heads/main/LSSTCam/bad.ecsv"
bad_visits = pd.read_csv(uri, comment="#")
bad_visit_list = bad_visits.exposure.to_list()
bad_visit_dayobs  = [v for v in bad_visit_list if str(v)[0:8] == day_obs.replace('-', '')]

In [ ]:
# Were there visits where the rotation angle was changed after the request/next visit?
changed_rot = vt[abs(vt.skyAngle - vt.sky_rotation) > 1]
if len(changed_rot) > 0:
    print(f"Visits with mismatch in sky_rotation (acquired rotation) compared to skyAngle in target request (+ nextvisit). - {len(changed_rot)}")
    display(changed_rot[cols])
else:
    print("No visits with mismatch in sky_rotation (acquired rotation) compared to skyAngle in target request (+ nextvisit).")

In [ ]:
rubintv_base = "https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam/event?key=lsstcam/"
def rubintv_links(day_obs, seq_num):
    witness_link = f"{rubintv_base}{day_obs}/witness_detector/{seq_num :06d}/lsstcam_witness_detector_{day_obs}_{seq_num :06d}.jpg"
    witness_html = f'<a href="{witness_link}" target="_blank" rel="noreferrer noopener">{witness_link}</a>'
    mosaic_link = f"{rubintv_base}{day_obs}/calexp_mosaic/{seq_num :06d}/lsstcam_calexp_mosaic_{day_obs}_{seq_num :06d}.jpg"
    mosaic_html = f'<a href="{mosaic_link}" target="_blank" rel="noreferrer noopener">{mosaic_link}</a>'
    return mosaic_html, witness_html

In [ ]:
pcols = ['visit_id', 'day_obs', 'seq_num', 'time_target', 'time_observation', 'obs_start', 'scriptSalIndex', 
          'target_name', 's_ra', 's_dec', 'sky_rotation', 'band', 'psf_sigma_median',
         'zero_point_1s', 'zero_point_1s_pred', 'science_program', 'target_name']

if len(vt) == 0:
    print("No observations.")

else:
    quicklook_missing = np.where(np.isnan(vt.zero_point_median) & (vt.visit_id > 0))[0]
    big_zp_offset = np.where((vt.zero_point_1s_pred.values - vt.zero_point_1s.values) > 1.5)[0]
    failed_obs = np.where(np.isnan(vt.time_observation.values) & (vt.visit_id > 0))[0]
    issues = np.concatenate([quicklook_missing, big_zp_offset, failed_obs])
    issues = np.sort(issues)
    issues = np.unique(issues)
    print(f"Found {len(issues)} images with potential issues out of {len(vt)} visits {(len(issues)/len(vt))*100 :.1f}%")
    
    print("From the bad visit list: bad visits from the list which are not flagged here: ")
    print(set(bad_visit_dayobs).difference(set(vt.iloc[issues]['visit_id'].values)))
    
    if len(issues) == 0:
        print("No obvious issues found.")
    else:
        display(HTML("<hr>"))
        display(HTML("Potential problems (please only report OBJECT img_types)"))
        display(HTML("<br>"))
        for idx in issues:
            problem_string = f"IMG_TYPE {vt.iloc[idx].img_type}. Flags:"
            if idx in failed_obs:
                problem_string += " no observation event,"
            if idx in quicklook_missing:
                problem_string += " no quicklook,"
            if idx in big_zp_offset:
                problem_string += " big zeropoint offset,"
            problem_string = problem_string[0:-1] + '.'
            if vt.iloc[idx].visit_id in bad_visit_list:
                problem_string += " Already in bad visit list."
            #problem_string = ", ".join(problem_string)
            display(pd.DataFrame(vt.iloc[idx][pcols]).T)
            print(problem_string)
            mosaic_html, witness_html = rubintv_links(day_obs, vt.iloc[idx].seq_num)
            display(HTML(mosaic_html))
            display(HTML(witness_html))
            display(HTML("<br>"))
        display(HTML("<hr>"))

In [ ]:
if len(failed_obs) > 0:
    efd_and_messages, ecols = scriptqueue.get_consolidated_messages(sunset, sunrise, endpoints)
    mtimes = pd.to_datetime(efd_and_messages.time)
    for idx in failed_obs:
        display(HTML("<hr>"))
        display(HTML("ScriptQueue info on visits missing observations (cause of failure?)"))
        display(HTML("<br>"))
        display(pd.DataFrame(vt.iloc[idx][pcols]).T)
        display(HTML("ScriptQueue Information with same scriptSalIndex"))
        fo = vt.iloc[idx]
        fo_time = pd.to_datetime(fo.time_target)
        mm = efd_and_messages.iloc[np.where(abs(mtimes - fo_time) < pd.Timedelta(1, unit='hr'))[0]]
        mm = mm.query('script_salIndex == @fo.scriptSalIndex')        
        display(HTML(mm[ecols].to_html()))
        display(Markdown("--"))

In [ ]:
# Targets which did not result in a visit
cols = ['visit_id', 'day_obs', 'seq_num', 'time_target', 'time_observation', 'time_nextvisit', 'groupId', 
        'ra', 'decl', 'skyAngle', 'filter', 'scriptSalIndex', 'note']
if len(vt) > 0:
    incomplete_target = vt.query('visit_id == 0')[cols]
    if len(incomplete_target) == 0:
        print("All requested targets resulted in visits.")
    else:
        print("Targets requested by not observed.")
        display(HTML(incomplete_target.to_html()))